In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [10]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# layer
apple_price = mul_apple_layer.forward(apple, apple_num)
apple_price = mul_tax_layer.forward(apple_price, tax)

print(apple_price)

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [11]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x + y
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

In [16]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_layer = AddLayer()
mul_tax_layer = MulLayer()

# forwad
mul_apple_price = mul_apple_layer.forward(apple, apple_num)
mul_orange_price = mul_orange_layer.forward(orange, orange_num)
add_apple_orange_price = add_layer.forward(mul_apple_price, mul_orange_price)
mul_tax_price = mul_tax_layer.forward(add_apple_orange_price, tax)

print(mul_tax_price)

# backward
dprice = 1
dprice_all, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorg_price = add_layer.backward(dprice_all)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorg, dorg_num = mul_orange_layer.backward(dorg_price)

print(dapple, dapple_num, dorg, dorg_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


In [17]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [21]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
relu = Relu()
relu.forward(x)

[[ 1.  -0.5]
 [-2.   3. ]]


array([[1., 0.],
       [0., 3.]])

In [27]:
class Sigmod:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out
        
    def backward(self, d):
        dx = d * (1 - self.out) * self.out
        
        return dx

In [31]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
s = Sigmod()
f = s.forward(x)
b = s.backward(x)
print("forward: ", f)
print("backward: ", b)

forward:  [[0.73105858 0.37754067]
 [0.11920292 0.95257413]]
backward:  [[ 0.19661193 -0.11750186]
 [-0.20998717  0.13552998]]


In [36]:
X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)
print(X)
print(X.shape)

[0.54199658 0.27371279]
(2,)


In [37]:
class Affine:
    def __init__(self, w, b):
        self.x = None
        self.W = w
        self.b = b
        self.dW = None
        self.db = None
    
    def forward(self, x):
        out = np.dot(x, self.W) + b
        self.x = x
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

In [38]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [40]:
# y代表学习结果，t代表真实标签
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [41]:
class SoftmaxWithLoss:
    def __init__(self):
        self.y = None
        self.t = None
        self.loss = None
    
    def forward(self, x, t):
        self.y = softmax(x)
        self.t = t
        # 这里为什么是y和t呢？y代表学习结果，t代笔真实的分类
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
        
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx